# Классификация спектров

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale as std_scale, robust_scale

from scipy.signal import find_peaks, find_peaks_cwt, peak_widths

import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale
import sklearn.metrics as metrics

import matplotlib.pyplot as plt
import matplotlib.animation as ani

import mlgrad.models as models
import mlgrad.regr as regr
import mlgrad.cls as cls
import mlgrad.plots as plots
import mlgrad.funcs as funcs
import mlgrad.funcs2 as funcs2
import mlgrad.loss as loss
import mlgrad.gd as gd
import mlgrad.smooth as smooth

import rampy
import pybaselines

In [2]:
def plot_spectra(d, ax=None):
    if ax is None:
        ax = plt.gca()
    for i, key in enumerate(d):
        Xs, Ys = d[key]
        if key == '191':
            color = 'b'
            label = "B"
        if key == '606':
            color = 'r'
            label = "A"
    
        ax.plot(np.mean(Ys, axis=0), color=color, linewidth=2.0, label=label)
    ax.legend(loc="upper left")
    ax.set_xlabel("$k$ (индекс точки)")
    ax.set_ylabel("Интенсивность (нормированная)")
    ax.set_title("Спектры (усредненные) по классам и значения коэффициентов")
    ax.minorticks_on()

def plot_params(param, ax=None):
    if ax is None:
        ax = plt.gca()
    pp = np.asarray(param)
    ymax = np.asarray(pp[1:].copy())
    ymin = np.asarray(pp[1:].copy())
    np.putmask(ymax, ymax<0, 0)
    np.putmask(ymin, ymax>0, 0)
    n = len(ymin)
    print(n)
    ax.vlines(range(n), ymin, ymax, color='k', linewidth=1., label=r"Вектор коэффициентов $\mathbf{w}$")
    ax.minorticks_on()
    ax.set_ylabel(r"$w_k$")
    ax.legend(loc="upper right")


In [3]:
def robust_mean(ys, tau=3.5, median=np.median):
    med = median(ys)
    dys = abs(ys - med)
    zz = 0.67449 * dys / median(dys)
    ii = (zz <= tau)
    return ys[ii].mean()

def robust_range(ys, tau=3.5, median=np.median):
    med = median(ys)
    dys = abs(ys - med)
    zz = 0.6745 * dys / median(dys)
    ii = (zz <= tau)
    return ys[ii]

def robust_indexes(ys, tau=3.5, median=np.median):
    med = median(ys)
    dys = abs(ys - med)
    zz = 0.67449 * dys / median(dys)
    return zz <= tau

def postprocess_data2(Ys, tau=3.5, median=np.median):
    N, m = Ys.shape
    RR = []

    if m % 2:
        flag = True
        i1, i2 = (m//2)-1, m//2
    else:
        flag = False
        i1 = (m//2) + 1

    for ys in Ys:
        ys.sort()
        if flag:
            med = (ys[i1] + ys[i2])/2
        else:
            med = ys[i1]
        dys = abs(ys - med)
        dys.sort()
        if flag:
            dys_med = (dys[i1] + dys[i2]) / 2
        else:
            dys_med = dys[i1]
        zs = 0.67449 * dys / dys_med
        RR.append(ys[zs <= tau])
    
    Y = np.empty(N, 'd')
    concatenate = np.concatenate

    Y[0] = RR[0].mean()
    Y[N-1] = RR[N-1].mean()
    
    for i in range(1, N-1):
        ys = RR[i]
        ys_min, ys_max = ys.min(), ys.max()

        ys1 = RR[i-1]
        ys2 = RR[i+1]
        
        ys1 = ys1[(ys_min <= ys1) & (ys1 <= ys_max)]        
        ys2 = ys2[(ys_min <= ys2) & (ys2 <= ys_max)]

        ys = concatenate((ys1,ys,ys2))
        Y[i] = ys.mean()
    
    return Y

def postprocess_data1(Ys, tau=3.5):
    return [robust_mean(ys, tau=tau) for ys in Ys]


In [4]:
def load_data(root):
    import os
    median = np.median
    d = {}
    for fold in os.listdir(root):
        Xs = []
        Ys = []
        for fname in os.listdir(f"{root}/{fold}"):
            if not fname.endswith(".txt"):
                continue
            with open(f"{root}/{fold}/{fname}", 'rt') as h:

                xy = np.loadtxt(f"{root}/{fold}/{fname}", delimiter="\t")
                xs = xy[:,0]
                ys = xy[:,1:]
                ys /= 1000

                # plt.figure(figsize=(10,3))
                # for y in ys:
                #     s = abs(y - median(y))
                #     s /= median(s)
                #     s *= 0.67449
                #     plt.plot(sorted(s), linewidth=0.5)
                # plt.hlines(3.5, 0, 30, color='k', linewidth=1.5)
                # plt.show()

                # ys = postprocess_data1(ys, tau=3.0)
                ys = postprocess_data2(ys, tau=3.0)

                # ys_sorted = np.sort(ys, axis=1)
                # ys = np.mean(ys_sorted[:,:27], axis=1)
                Xs.append(xs[42:])
                Ys.append(ys[42:])

        Xs, Ys =  np.array(Xs), np.array(Ys)

        # med = np.median(Ys, axis=0)
        # dd = abs(Ys - med)
        # ss = dd.sum(axis=1)
        # ii = robust_indexes(ss, tau=3.5)
        # Xs, Ys = Xs[ii,:], Ys[ii,:]
        
        if '191' in fold:
            d['191'] = (Xs, Ys)
        if '606' in fold:
            d['606'] = (Xs, Ys)
            
    return d

In [5]:
d = load_data("Table")

for i, key in enumerate(d):
    nn = len(d)
    Xs, Ys = d[key]
    if key == '191':
        color = 'b'
        title = "B"
    if key == '606':
        color = 'r'
        title = "A"

    plt.figure(figsize=(15,7))
    for k in range(len(Xs)):
        xs = Xs[k]
        ys = Ys[k]
        ys = smooth.whittaker(ys, tau2=10.0, h=0.001, func=funcs.Square())
        # ys = rampy.smooth(xs, ys, method="whittaker", Lambda=4.0)
        # bs, _ = pybaselines.whittaker.aspls(ys, lam=1.0e5)
        # bs, _ = pybaselines.morphological.mor(ys)
        # ys = ys - bs
        # ys = smooth.whittaker(ys, tau2=10.0)
        ys -= ys.min()
        ys /= np.trapz(ys, xs)
        ys *= 1000
        Ys[k,:] = ys
        
        # plt.plot(ys, color=color, linewidth=0.2, marker='s', markersize=2)
        plt.plot(ys, color=color, linewidth=0.2)    
        
    plt.plot(np.median(Ys, axis=0), color='k', linewidth=2.0)
    plt.hlines(0, -10, 1000, linewidth=0.5, colors='k', linestyle='--')
    # plt.xlim(100, 300)
    plt.minorticks_on()
    plt.xlabel("Индекс (длины волны)")
    plt.ylabel("Интенсивность (нормированная)")
    plt.title(f"Класс: {title}")
    plt.grid(1)
    plt.tight_layout()
    plt.show()

    # plt.figure(figsize=(10,3))
    # plt.title(f"Класс: {title}")
    # ys_med = 
    # for y in Ys:
    #     s = abs(y - np.median(y))
    #     z = 0.67449 * s / np.median(s)
    #     plt.plot(sorted(z), linewidth=0.5)
    # plt.hlines(3.5, 0, 30, color=color, linewidth=1.5)
    # plt.show()


TypeError: 'module' object is not callable

<Figure size 1500x700 with 0 Axes>

In [ ]:
for i, key in enumerate(d):
    Xs, Ys = d[key]
    if key == '606': # А
        XX2 = Ys
        YY2 = np.full(len(Ys), +1, 'd')
        print(f"Класс A: {len(Ys)} спектров")
    if key == '191': # B
        XX1 = Ys
        YY1 = np.full(len(Ys), -1, 'd')
        print(f"Класс B: {len(Ys)} спектров")

XX = np.concatenate((XX1, XX2), axis=0) 
YY = np.concatenate((YY1, YY2), axis=0) 
print(f"Всего {len(XX)} спектров")

print("Класс A: метка +1")
print("Класс B: метка -1")

XX = np.ascontiguousarray(XX)
YY = np.ascontiguousarray(YY)

N, n = XX.shape
print(r"Размерность вектора x:", n)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca1 = PCA(2)
pca1.fit(XX1.T)
U1 = pca1.transform(XX1.T)
print(U1.shape, YY1.shape)

pca2 = PCA(2)
pca2.fit(XX2.T)
U2 = pca2.transform(XX2.T)
print(U2.shape, YY2.shape)

plt.figure(figsize=(5,4))
plt.scatter(U1[:,0], U1[:,1], label='1')
plt.scatter(U2[:,0], U2[:,1], label='2')
plt.legend()
plt.show()

In [ ]:
pca1 = PCA(2)
pca1.fit(XX1)
U1_1 = pca1.transform(XX1)
U1_2 = pca1.transform(XX2)
pca2 = PCA(2)
pca2.fit(XX2)
U2_1 = pca2.transform(XX1)
U2_2 = pca2.transform(XX2)

plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.scatter(U1_1[:,0], U1_1[:,1], c='r')
plt.scatter(U1_2[:,0], U1_2[:,1], c='b')
plt.subplot(1,2,2)
plt.scatter(U2_1[:,0], U2_1[:,1], c='r')
plt.scatter(U2_2[:,0], U2_2[:,1], c='b')
plt.tight_layout()
plt.show()

In [ ]:
hinge_loss = loss.MarginLoss(funcs.Hinge(1.0))
l2_reg = funcs2.SquareNorm()
l1_reg = funcs2.AbsoluteNorm()
soft_l1_reg = funcs2.SoftAbsoluteNorm(0.001)
sigmoidal_func = funcs.Sigmoidal(6.0)
sigmoidal_func2 = funcs.Sigmoidal(2.0)
sq_loss = loss.ErrorLoss(funcs.Square())

### Линейное разделение (функция Хинжа, без регуляризации)

$$
w^\star = \arg \min_{\mathbf{w}} 
\frac{1}{N}\sum_{k=1}^N 
H\bigl(y_k[w_0+(\mathbf{w},\mathbf{x}_k)]\bigr)
$$
$$
H(u) = \max(-u,0)
$$

In [ ]:
# График функции Хинжа
plt.figure(figsize=(6,3))
plt.plot([-1,0,1],[1,0,0], linewidth=2, color='k')
plt.title("Функция Хинжа")
plt.hlines(0,-1,0, linestyles='dashed', color='LightGrey')
plt.vlines(0,0,1, linestyles='dashed', color='LightGrey')
plt.xlabel(r"$u$")
plt.ylabel(r"$H(u)$")
plt.show()

In [ ]:
mod1 = models.LinearModel(n) # линейная модель

In [ ]:
alg1 = cls.classification_as_regr(XX, YY, mod1,
                                  lossfunc=hinge_loss, 
                                  normalizer=gd.Masked(mod1.n_param, 1.0e-5),
                                  h=0.001, n_iter=10000)

plt.figure(figsize=(12,4))
plots.plot_losses(alg1)
plt.show()

print("Относительное число правильных ответов:", 
        metrics.accuracy_score(YY1, np.sign(mod1.evaluate_all(XX1))),
        metrics.accuracy_score(YY2, np.sign(mod1.evaluate_all(XX2))))

In [ ]:
plt.figure(figsize=(12,5))
plots.plot_cls_function(mod1, XX, YY)
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(mod1.param, ax=ax2)
plt.show()

### Линейное разделение (функция Хинжа, L2 регуляризация)

$$
w^\star = \arg \min_{\mathbf{w}} \frac{1}{N} \sum_{k=1}^N 
H\bigl(y_k[w_0+(\mathbf{w},\mathbf{x}_k)]\bigr) + \tau\|\mathbf{w}\|_2^2,
$$
$$
H(u) = \max(-u,0)
$$
$$
\|\mathbf{w}\|_2^2 = w_1^2+\cdots+w_n^2
$$

In [ ]:
mod2 = models.LinearModel(n)

In [ ]:
alg2 = cls.classification_as_regr(XX, YY, mod2,
                                 lossfunc=hinge_loss, 
                                 regnorm=l2_reg, tau=0.4,
                                 normalizer=gd.Masked(mod2.n_param, 1.0e-5),
                                 h=0.001, n_iter=10000)
plt.figure(figsize=(12,5))
plots.plot_losses(alg2)
plt.show()

print("Относительное число правильных ответов:", 
        metrics.accuracy_score(YY1, np.sign(mod2.evaluate_all(XX1))),
        metrics.accuracy_score(YY2, np.sign(mod2.evaluate_all(XX2))))

In [ ]:
plt.figure(figsize=(12,5))
plots.plot_cls_function(mod2, XX, YY)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(mod2.param, ax=ax2)
plt.show()

### Линейное разделение (функция Хинжа, L1 регуляризацией)

$$
w^\star = \arg \min_{\mathbf{w}} \frac{1}{N} \sum_{k=1}^N 
H\bigl(y_k[w_0+(\mathbf{w},\mathbf{x}_k)]\bigr) + 
\tau\|\mathbf{w}\|_1,
$$
$$
H(u) = \max(-u,0)
$$
$$
\|\mathbf{w}\|_1 = |w_1|+\cdots+|w_n|
$$

In [ ]:
mod3 = models.LinearModel(n)

In [ ]:
alg3 = cls.classification_as_regr(XX, YY, mod3,
                                 lossfunc=hinge_loss, 
                                 regnorm=l1_reg, tau=0.005,
                                 normalizer=gd.Masked(mod3.n_param, 1.0e-5),
                                 h=0.001, n_iter=10000)
plt.figure(figsize=(12,5))
plots.plot_losses(alg3)
plt.show()

print("Относительное число правильных ответов:", 
        metrics.accuracy_score(YY1, np.sign(mod3.evaluate_all(XX1))),
        metrics.accuracy_score(YY2, np.sign(mod3.evaluate_all(XX2))))

print((mod3.param.base != 0).sum())

In [ ]:
plt.figure(figsize=(12,5))
plots.plot_cls_function(mod3, XX, YY)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(mod3.param, ax=ax2)
plt.show()

In [ ]:
counts = np.zeros(XX.shape[1], 'i')
qwals = []
for i in range(50):
    _mod = models.LinearModel(n)
    for j in range(2):
        _alg = cls.classification_as_regr(XX, YY, _mod,
                                     lossfunc=hinge_loss, 
                                     regnorm=l1_reg, tau=0.003,
                                     normalizer=gd.Masked(_mod.n_param, 1.0e-5),
                                     h=0.001, n_iter=10000)
    _qwal = metrics.accuracy_score(YY, np.sign(_mod.evaluate_all(XX)))
    qwals.append(_qwal)
    
    I = np.argwhere(abs(_alg.risk.param.base[1:]) >= 0.001)[:,0]
    counts[I] += 1

# print(counts)

In [ ]:
plt.figure(figsize=(15,8))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(counts, ax=ax2)
plt.show()

plt.hist(qwals, bins=10)
plt.show()

In [ ]:
I1 = np.argwhere(counts > 5)[:,0]
XXI = np.ascontiguousarray(XX[:,I1])
print(XXI.shape, YY.shape)

In [ ]:
mod3_i = models.LinearModel(XXI.shape[1])

In [ ]:
alg3_i = cls.classification_as_regr(XXI, YY, mod3_i,
                                 lossfunc=hinge_loss, 
                                 regnorm=l1_reg, tau=0.003,
                                 normalizer=gd.Masked(mod3_i.n_param, 1.0e-5),
                                 h=0.001, n_iter=10000)
plt.figure(figsize=(12,5))
plots.plot_losses(alg3_i)
plt.show()

print("Относительное число правильных ответов:", 
        metrics.accuracy_score(YY1, np.sign(mod3_i.evaluate_all(np.ascontiguousarray(XX1[:,I1])))),
        metrics.accuracy_score(YY2, np.sign(mod3_i.evaluate_all(np.ascontiguousarray(XX2[:,I1])))))

print((mod3_i.param.base != 0).sum())

In [ ]:
p = np.zeros(XX.shape[1]+1, 'd')
p[I1] = mod3_i.param[1:]
p[0] = mod3_i.param[0]

plt.figure(figsize=(15,8))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(p, ax=ax2)
plt.show()


## Линейное разделение (сигмоидальная функция)

$$
y = \sigma\bigl(w_0 + (\mathbf{w},\mathbf{x})\bigr)
$$
где
$$
\sigma(s) = \mathrm{th}(s) = \frac{e^{\alpha x} - e^{-\alpha x}}{e^{\alpha x} + e^{-\alpha x}}
$$

$$
w^\star = \arg \min_{\mathbf{w}} \frac{1}{N} \sum_{k=1}^N 
\bigl(\sigma(w_0+(\mathbf{w},\mathbf{x}_k)) - y_k\bigr)^2 
$$


In [ ]:
sigmoidal_func = funcs.Sigmoidal(6.0)
mod1_s = models.SigmaNeuronModel(sigmoidal_func, n)

In [ ]:
alg1_s = cls.classification_as_regr(XX, YY, mod1_s,
                                 lossfunc=sq_loss,
                                 normalizer=gd.Masked(mod1_s.n_param, 1.0e-4),
                                 h=0.001, n_iter=5000)
plt.figure(figsize=(12,3))
plt.plot(alg1_s.lvals)
plt.show()

print("Относительное число правильных ответов:", 
      metrics.accuracy_score(YY, np.sign(mod1_s.evaluate_all(XX))))

In [ ]:
plt.figure(figsize=(12,5))
plots.plot_cls_function(mod1_s, XX, YY)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(mod1_s, ax=ax2)
plt.show()

### Линейное разделение (максимизация отступа, с L2 регуляризацией)

$$
w^\star = \arg \min_{\mathbf{w}} \frac{1}{N} \sum_{k=1}^N 
\bigl(\sigma(w_0+(\mathbf{w},\mathbf{x}_k)) - y_k\bigr)^2 + 
\tau\|\mathbf{w}\|_2^2,
$$
$$
\|\mathbf{w}\|_2^2 = w_1^2+\cdots+w_n^2
$$

In [ ]:
mod2_s = models.SigmaNeuronModel(sigmoidal_func, n)

In [ ]:
alg2_s = cls.classification_as_regr(XX, YY, mod2_s,
                                 lossfunc=sq_loss, 
                                 regnorm=soft_l1_reg, tau=0.1,
                                 normalizer=gd.Masked(mod2_s.n_param, 1.0e-5),
                                 h=0.001, n_iter=5000)
plt.figure(figsize=(12,5))
plots.plot_losses(alg2_s)
plt.show()

metrics.accuracy_score(YY, np.sign(mod2_s.evaluate_all(XX)))

In [ ]:
plt.figure(figsize=(12,5))
plots.plot_cls_function(mod2_s, XX, YY)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(mod2_s.param, ax=ax2)
plt.show()

### Линейное разделение (максимизация отступа, с L1 регуляризацией)

$$
w^\star = \arg \min_{\mathbf{w}} \frac{1}{N} \sum_{k=1}^N 
\sigma\bigl(y_k[w_0+(\mathbf{w},\mathbf{x}_k)]\bigr) + \tau\|\mathbf{w}\|_1,
$$
$$
\|\mathbf{w}\|_1 = |w_1|+\cdots+|w_n|
$$

In [ ]:
mod3_s = models.SigmaNeuronModel(sigmoidal_func2, n)

In [ ]:
alg3_s = cls.classification_as_regr(XX, YY, mod3_s,
                                 lossfunc=sq_loss, 
                                 regnorm=l1_reg, tau=0.01,
                                 normalizer=gd.Masked(mod3_s.n_param, 1.0e-5),
                                 h=0.001, n_iter=10000, n_restart=2)
plt.figure(figsize=(12,5))
plots.plot_losses(alg3_s)
plt.show()

print("Относительное число правильных ответов:", 
      metrics.accuracy_score(YY1, np.sign(mod3_s.evaluate_all(XX1))),
      metrics.accuracy_score(YY2, np.sign(mod3_s.evaluate_all(XX2))),
      metrics.accuracy_score(YY, np.sign(mod3_s.evaluate_all(XX))),)

print((mod3_s.param.base != 0).sum())

In [ ]:
plt.figure(figsize=(12,5))
plots.plot_cls_function(mod3_s, XX, YY)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(mod3_s.param, ax=ax2)
plt.show()

In [ ]:
counts = np.zeros(XX.shape[1], 'i')
qwals = []
for i in range(50):
    _mod = models.SigmaNeuronModel(sigmoidal_func2, n)
    _alg = cls.classification_as_regr(XX, YY, _mod,
                                     lossfunc=sq_loss, 
                                     regnorm=l1_reg, tau=0.01,
                                     normalizer=gd.Masked(_mod.n_param, 1.0e-5),
                                     h=0.001, n_iter=10000, n_restart=2)
    _qwal = metrics.accuracy_score(YY, np.sign(_mod.evaluate_all(XX)))
    qwals.append(_qwal)
    
    I = np.argwhere(abs(_alg.risk.param.base[1:]) >= 0.001)[:,0]
    counts[I] += 1

# print(counts)

In [ ]:
plt.figure(figsize=(15,8))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(counts, ax=ax2)
plt.show()

In [ ]:
plt.hist(qwals, rwidth=0.9)
plt.show()

In [ ]:
I1 = np.argwhere(counts > 5)[:,0]
XXI = np.ascontiguousarray(XX[:,I1])
print(XXI.shape, YY.shape)

In [ ]:
mod3_s1 = models.SigmaNeuronModel(sigmoidal_func2, XXI.shape[1])
alg3_s1 = cls.classification_as_regr(XXI, YY, mod3_s1,
                                 lossfunc=sq_loss, 
                                 regnorm=l1_reg, tau=0.005,
                                 normalizer=gd.Masked(mod3_s1.n_param, 1.0e-5),
                                 h=0.001, n_iter=10000, n_restart=1)
plt.figure(figsize=(12,5))
plots.plot_losses(alg3_s1)
plt.show()

print("Относительное число правильных ответов:", 
      metrics.accuracy_score(YY1, np.sign(mod3_s1.evaluate_all(np.ascontiguousarray(XX1[:,I1])))),
      metrics.accuracy_score(YY2, np.sign(mod3_s1.evaluate_all(np.ascontiguousarray(XX2[:,I1])))),
      metrics.accuracy_score(YY, np.sign(mod3_s1.evaluate_all(XXI))),)

print((mod3_s1.param.base != 0).sum())

In [ ]:
p = np.zeros(XX.shape[1]+1, 'd')
p[I1] = mod3_s1.param[1:]
p[0] = mod3_s1.param[0]

plt.figure(figsize=(15,8))
ax1 = plt.gca()
plot_spectra(d, ax=ax1)
ax2 = plt.twinx(ax1)
plot_params(p, ax=ax2)
plt.show()


In [ ]:
n_hidden = 10
linear4 = models.LinearLayer(XX.shape[1], n_hidden)
scaler4 = models.ScaleLayer(sigmoidal_func, n_hidden)
body4 = models.FFNetworkModel()
body4.add(linear4)
body4.add(scaler4)
head4 = models.LinearModel(n_hidden)
mod4_s = models.FFNetworkFuncModel(head4, body4)
mod4_s.allocate()
mod4_s.init_param()

In [ ]:
alg4_s = cls.classification_as_regr(XX, YY, mod4_s,
                                 lossfunc=sq_loss, 
                                 regnorm=l1_reg, tau=0.01,
                                 normalizer=gd.Masked(mod4_s.n_param, 1.0e-5),
                                 h=0.001, n_iter=10000)
plt.figure(figsize=(12,5))
plots.plot_losses(alg4_s)
plt.show()

print("Относительное число правильных ответов:", 
      metrics.accuracy_score(YY1, np.sign(mod4_s.evaluate_all(XX1))),
      metrics.accuracy_score(YY2, np.sign(mod4_s.evaluate_all(XX2))))

In [ ]:
plt.figure(figsize=(12,5))
plots.plot_cls_function(mod4_s, XX, YY)
plt.show()

In [ ]:
plt.figure(figsize=(20,4))
plot_params(mod4_s)
plt.vlines([n_hidden+XX.shape[1]*i for i in range(n_hidden)], 0,4)
plt.show()

In [ ]:
n_hidden = 7
linear5 = models.LinearLayer(XX.shape[1], n_hidden)
scaler5 = models.ScaleLayer(sigmoidal_func, n_hidden)
body5 = models.FFNetworkModel()
body5.add(linear5)
body5.add(scaler5)
head5 = models.LinearModel(n_hidden)
mod5_s = models.FFNetworkFuncModel(head5, body5)
mod5_s.allocate()
mod5_s.init_param()

In [ ]:
alg5_s = cls.classification_as_regr(XX, YY, mod5_s,
                                 lossfunc=hinge_loss, 
                                 regnorm=l1_reg, tau=0.001,
                                 normalizer=gd.Masked(mod5_s.n_param, 1.0e-4),
                                 h=0.001, n_iter=10000, n_restart=3)
plt.figure(figsize=(12,5))
plots.plot_losses(alg5_s)
plt.show()

print("Относительное число правильных ответов:", 
      metrics.accuracy_score(YY1, np.sign(mod5_s.evaluate_all(XX1))),
      metrics.accuracy_score(YY2, np.sign(mod5_s.evaluate_all(XX2))))

In [ ]:
plt.figure(figsize=(12,5))
plots.plot_cls_function(mod5_s, XX, YY)
plt.show()

In [ ]:
plt.figure(figsize=(20,4))
plot_params(mod5_s)
plt.vlines([n_hidden+(XX.shape[1]+1)*i for i in range(n_hidden)], 0,3)
plt.show()